In [ ]:
from langsmith import traceable
from langsmith import Client
import os
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "My Research Agent"  
client = Client()



In [ ]:
import operator
from dataclasses import dataclass, field
from typing import Literal, List, Optional, TypedDict
from typing import TypedDict, List, Literal, Optional
from typing import TypedDict, List, Literal, Optional

#state for our assistant: exposed to user
class SummaryState(TypedDict):
    research_topic: str
    search_query: Optional[str]
    sources_gathered: List[str]
    web_research_results: List[str]
    running_summary: Optional[str]
    research_loop_count: int

#exposed to user
class SummaryStateInput(TypedDict):
    research_topic: str

#exposed to user
class SummaryStateOutput(TypedDict):
    running_summary: str
    sources_gathered: List[str]




In [ ]:
from tavily import TavilyClient
import os
from langchain_ollama import ChatOllama

### LLM
local_llm = "deepseek-r1:14b"
# local_llm = "qwen2.5:14b"
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json")


### Search
tavily_client = TavilyClient(api_key="")

@traceable
def tavily_search(query, include_raw_content=True, max_results=3):
    """Search the web using Tavily API with error handling"""
    try:
        response = tavily_client.search(
            query=query,
            max_results=max_results,
            include_raw_content=include_raw_content
        )
        # Validate response structure
        if not isinstance(response.get("results"), list):
            print("Invalid Tavily response format:", response)
            return {"results": []}
        return response
    except Exception as e:
        print(f"Tavily search failed: {e}")
        return {"results": []}


In [ ]:

# !brew install graphviz
import json
from langchain_core.runnables import RunnableConfig
from langgraph.graph import START, END, StateGraph
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.runnables import RunnableConfig

# from langgraph.render.render import draw_mermaid

### Query Writer 
query_writer_instructions="""
You are a query generation assistant. Your task is to create a targeted web search query to gather information on a specific research topic.

Here is the research topic you'll be working with:
{research_topic}

To generate an effective query, follow these steps:
Analyze the given research topic carefully.
Identify a specific aspect or subtopic within the main topic that would benefit from further research.
Formulate a clear and concise search query that will yield relevant results for this aspect.
Consider using advanced search operators if appropriate (e.g., quotation marks for exact phrases, site: for specific websites, etc.).
Your output should be a JSON object with the following structure:
{
"query": "Your generated search query",
"aspect": "The specific aspect or subtopic you're focusing on",
"rationale": "A brief explanation of why you chose this query and how it relates to the research topic"
}
Here's an example of how your output should look:
{
"query": ""renewable energy" wind farms efficiency",
"aspect": "Wind energy efficiency",
"rationale": "This query focuses on the efficiency of wind farms, which is a crucial aspect of renewable energy. It will help gather information on the current state and potential improvements in wind energy technology."
}
Remember to tailor your query to the specific research topic provided and ensure it's designed to gather relevant and useful information.
"""

summarizer_instructions="""

You are tasked with generating a high-quality summary of web search results. Your approach will depend on whether you're extending an existing summary or creating a new one. Follow these instructions carefully:
First, review the following inputs:
<existing_summary>
{$EXISTING_SUMMARY}
</existing_summary>
<search_results>
{$SEARCH_RESULTS}
</search_results>
<summary_type>
{$SUMMARY_TYPE}
</summary_type>
If the summary_type is "EXTEND", follow these steps:
Carefully read the existing summary to understand its content, style, and depth.
Analyze the new search results to identify information that is not already covered in the existing summary.
Seamlessly integrate the new, non-redundant information into the existing summary.
Maintain consistency with the existing content's style and level of detail.
Ensure smooth transitions between existing and new content.
Do not repeat information that is already present in the existing summary.
If the summary_type is "NEW", follow these steps:
Thoroughly review all provided search results.
Identify the most relevant information from each source related to the report topic.
Organize the key points to create a coherent flow of information.
Highlight significant findings or insights from the search results.
Provide a concise yet comprehensive overview of the topic.
For both summary types, adhere to these guidelines:
Focus on factual, objective information from the search results.
Maintain a consistent technical depth throughout the summary.
Avoid redundancy and repetition of information.
Do not use phrases like "based on the new results" or "according to additional sources".
Do not add a preamble like "Here is an extended summary..." or "This is a new summary...".
Do not include a References or Works Cited section.
Output your summary directly, without any introductory statements. Use clear, concise language and ensure a logical flow of information. The summary should stand alone as a coherent piece of writing, regardless of whether it's an extension or a new creation.
Provide your summary within <summary> tags.
"""

reflection_instructions = """You are an expert research assistant tasked with analyzing a summary about a specific research topic. Your goal is to identify knowledge gaps and generate a follow-up question to expand understanding of the topic. Follow these instructions carefully:
Read and analyze the following research topic:
<research_topic>
{research_topic}
</research_topic>
Now, carefully read and analyze the following summary related to the research topic:
<summary> {summary} </summary>
Analyze the summary to identify knowledge gaps or areas that need deeper exploration. Focus on:
Technical details that may be missing or unclear
Implementation specifics that could be elaborated upon
Emerging trends or recent developments that might not be fully covered
Potential limitations or challenges not addressed in the summary
Based on your analysis, generate a follow-up question that would help expand understanding of the topic. Ensure that the question:
Addresses a significant knowledge gap or area needing deeper exploration
Is specific and focused on technical details, implementation specifics, or emerging trends
Is self-contained and includes necessary context for a web search
Can lead to meaningful insights or advancements in the research topic
Format your response as a JSON object with the following structure:
{
"knowledge_gap": "Briefly describe the identified knowledge gap or area needing deeper exploration",
"follow_up_query": "Your generated follow-up question"
}
Ensure that both the knowledge gap description and the follow-up query are clear, concise, and directly related to the research topic and summary provided."""


# @traceable
def generate_query(state: SummaryState):
    """ Generate a query for web search """
    visualizer.show(active_node="generate_query")
    # Format the prompt
    # query_writer_instructions_formatted = query_writer_instructions.format(research_topic=state.research_topic)
    query_writer_instructions_formatted = query_writer_instructions.format(
    research_topic=state["research_topic"]  # ✅
)

    # Generate a query
    result = llm_json_mode.invoke(
        [SystemMessage(content=query_writer_instructions_formatted),
        HumanMessage(content=f"Generate a query for web search:")]
    )   
    query = json.loads(result.content)
    
    return {"search_query": query['query']}

# @traceable
def web_research(state: SummaryState):
    """ Gather information from the web """
    visualizer.show(active_node="web_research")
    search_results = tavily_search(state["search_query"], include_raw_content=True, max_results=3)
    search_str = deduplicate_and_format_sources(search_results)  # ✅ Pass full response
    return {
        "sources_gathered": [format_sources(search_results["results"])],  # Access 'results' key
        "research_loop_count": state["research_loop_count"] + 1,
        "web_research_results": [search_str]
    }

# @traceable
def summarize_sources(state: SummaryState):
    """ Summarize the gathered sources """
    visualizer.show(active_node="summarize_sources")
    
    
    # Existing summary
    existing_summary = state["running_summary"]

    # Most recent web research
    most_recent_web_research = state["web_research_results"][-1]

    # Build the human message
    if existing_summary:
        human_message_content = (
            f"Extend the existing summary: {existing_summary}\n\n"
            f"Include new search results: {most_recent_web_research} "
            f"That addresses the following topic: {state["research_topic"]}"
        )
    else:
        human_message_content = (
            f"Generate a summary of these search results: {most_recent_web_research} "
            f"That addresses the following topic: {state["research_topic"]}"
        )

    # Run the LLM
    result = llm.invoke(
        [SystemMessage(content=summarizer_instructions),
        HumanMessage(content=human_message_content)]
    )

    running_summary = result.content
    return {"running_summary": running_summary}

# @traceable
def reflect_on_summary(state: SummaryState):
    """ Reflect on the summary and generate a follow-up query """
    visualizer.show(active_node="reflect_on_summary")
    

    # Generate a query
    result = llm_json_mode.invoke(
        [SystemMessage(content=reflection_instructions.format(research_topic=state["research_topic"])),
        HumanMessage(content=f"Identify a knowledge gap and generate a follow-up web search query based on our existing knowledge: {state["running_summary"]}")]
    ) 
    follow_up_query = json.loads(result.content)

    # Overwrite the search query
    return {"search_query": follow_up_query['follow_up_query']}

# @traceable
def finalize_summary(state: SummaryState):
    """ Finalize the summary """
    visualizer.show(active_node="finalize_summary")
    
    
    # Format all accumulated sources into a single bulleted list
    all_sources = "\n".join(source for source in state["sources_gathered"])
    state["running_summary"] = f"## Summary\n\n{state["running_summary"]}\n\n ### Sources:\n{all_sources}"
    return {"running_summary": state["running_summary"]}


# @traceable
def route_research(state: SummaryState, config: RunnableConfig) -> Literal["finalize_summary", "web_research"]:
    """ Route the research based on the follow-up query """
    
    visualizer.show(active_node="route_research")
    
    max_loops = config.get("configurable", {}).get("max_web_research_loops", 3)  # Default to 3 if not specified
    if state["research_loop_count"] <= max_loops:
        return "web_research"
    else:
        return "finalize_summary"
    

# Add nodes and edges
builder = StateGraph(SummaryState)#, input=SummaryStateInput, output=SummaryStateOutput, config_schema=RunnableConfig)
builder.add_node("generate_query", generate_query, metadata={"color": "lightblue"})
builder.add_node("web_research", web_research, metadata={"color": "lightgreen"})
builder.add_node("summarize_sources", summarize_sources, metadata={"color": "lightblue"})
builder.add_node("reflect_on_summary", reflect_on_summary, metadata={"color": "lightgreen"})
builder.add_node("finalize_summary", finalize_summary, metadata={"color": "lightblue"})

# Add edges
builder.add_edge(START, "generate_query")
builder.add_edge("generate_query", "web_research")
builder.add_edge("web_research", "summarize_sources")
builder.add_edge("summarize_sources", "reflect_on_summary")
builder.add_conditional_edges("reflect_on_summary", route_research)
builder.add_edge("finalize_summary", END)

graph = builder.compile(debug=True)

# Test with proper state initialization
research_input = {
    "research_topic": "how many match centuries has virat kohli scored in 2024?",
    "search_query": None,
    "sources_gathered": [],
    "web_research_results": [],
    "running_summary": None,
    "research_loop_count": 0
}

try:
    summary = graph.invoke(
    research_input,
    config={"configurable": {"max_web_research_loops": 3}}
)
finally:
    visualizer.show()  #  
summary = graph.invoke(research_input)
print("FINAL SUMMARY:\n", summary["running_summary"])
print("\nSOURCES:\n", "\n".join(summary["sources_gathered"]))

[-1:checkpoint] State at the end of step -1:
{}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'research_loop_count': 0,
 'research_topic': 'how many match centuries has virat kohli scored in 2024?',
 'running_summary': None,
 'search_query': None,
 'sources_gathered': [],
 'web_research_results': []}
[0:writes] Finished step 0 with writes to 6 channels:
- research_topic -> 'how many match centuries has virat kohli scored in 2024?'
- search_query -> None
- sources_gathered -> []
- web_research_results -> []
- running_summary -> None
- research_loop_count -> 0
[0:checkpoint] State at the end of step 0:
{'research_loop_count': 0,
 'research_topic': 'how many match centuries has virat kohli scored in 2024?',
 'running_summary': None,
 'search_query': None,
 'sources_gathered': [],
 'web_research_results': []}
[1:tasks] Starting 1 task for step 1:
- generate_query -> {'research_loop_count': 0,
 'research_topic': 'how many match centuries has virat kohli scored in 2024?',
 'running_s

[1:writes] Finished step 1 with writes to 1 channel:
- search_query -> 'Virat Kohli centuries scored in 2024'
[1:checkpoint] State at the end of step 1:
{'research_loop_count': 0,
 'research_topic': 'how many match centuries has virat kohli scored in 2024?',
 'running_summary': None,
 'search_query': 'Virat Kohli centuries scored in 2024',
 'sources_gathered': [],
 'web_research_results': []}
[2:tasks] Starting 1 task for step 2:
- web_research -> {'research_loop_count': 0,
 'research_topic': 'how many match centuries has virat kohli scored in 2024?',
 'running_summary': None,
 'search_query': 'Virat Kohli centuries scored in 2024',
 'sources_gathered': [],
 'web_research_results': []}


[2:writes] Finished step 2 with writes to 3 channels:
- sources_gathered -> ['* Virat Kohli Total Centuries List [Updated 2024] - cricpoint\n'
 '  URL: https://cricpoint.in/virat-kohli-total-centuries-list/\n'
 '  Excerpt: Significantly, Kohli has the most international centuries among '
 'active players worldwide. Legendary player Sachin Tendulkar leads the list '
 'with 100 international centuries. While Virat Kohli has score...\n'
 '\n'
 '* Virat Kohli Total Centuries in All Format (2025 Stats)\n'
 '  URL: https://www.thecricketpanda.com/virat-kohli-centuries/\n'
 '  Excerpt: Virat Kohli Centuries In Every Format Virat Kohli’s batting '
 'excellence is evident in his century-making ability across all formats of '
 'international cricket. Centuries By Virat Kohli In IPL Kohli’s IPL...\n'
 '\n'
 '* Virat Kohli Year-Wise ODI & Test Runs (Year-by-Year Performance ...\n'
 '  URL: https://www.cricindeed.com/virat-kohli-performance-year-by-year/\n'
 "  Excerpt: CricIndeed For Cricket's Ama

[3:writes] Finished step 3 with writes to 1 channel:
- running_summary -> ('Virat Kohli has not played any international cricket matches in 2024, as the '
 'year is still ongoing and no official matches have taken place. Therefore, '
 'he has not scored any centuries in 2024 yet.')
[3:checkpoint] State at the end of step 3:
{'research_loop_count': 1,
 'research_topic': 'how many match centuries has virat kohli scored in 2024?',
 'running_summary': 'Virat Kohli has not played any international cricket '
                    'matches in 2024, as the year is still ongoing and no '
                    'official matches have taken place. Therefore, he has not '
                    'scored any centuries in 2024 yet.',
 'search_query': 'Virat Kohli centuries scored in 2024',
 'sources_gathered': ['* Virat Kohli Total Centuries List [Updated 2024] - '
                      'cricpoint\n'
                      '  URL: '
                      'https://cricpoint.in/virat-kohli-total-centuries-list/

[4:writes] Finished step 4 with writes to 1 channel:
- search_query -> ('"Virat Kohli match centuries 2024" + site:cricketarchive.com OR "Virat Kohli '
 'international cricket stats 2024" + filetype:pdf')
[4:checkpoint] State at the end of step 4:
{'research_loop_count': 1,
 'research_topic': 'how many match centuries has virat kohli scored in 2024?',
 'running_summary': 'Virat Kohli has not played any international cricket '
                    'matches in 2024, as the year is still ongoing and no '
                    'official matches have taken place. Therefore, he has not '
                    'scored any centuries in 2024 yet.',
 'search_query': '"Virat Kohli match centuries 2024" + site:cricketarchive.com '
                 'OR "Virat Kohli international cricket stats 2024" + '
                 'filetype:pdf',
 'sources_gathered': ['* Virat Kohli Total Centuries List [Updated 2024] - '
                      'cricpoint\n'
                      '  URL: '
                      'htt

...

[10:writes] Finished step 10 with writes to 1 channel:
- search_query -> ("What are the projected match century targets for Virat Kohli during India's "
 'upcoming series against Australia in 2024, and how do experts expect his '
 'performance to impact his overall international record?')
[10:checkpoint] State at the end of step 10:
{'research_loop_count': 3,
 'research_topic': 'how many match centuries has virat kohli scored in 2024?',
 'running_summary': 'Virat Kohli has not played any international cricket '
                    'matches in 2024, as the year is still ongoing and no '
                    'official matches have taken place. Therefore, he has not '
                    'scored any centuries in 2024 yet. However, according to '
                    "ESPN Cricinfo, Virat Kohli's performance in 2024 is "
                    'expected to be closely monitored by fans and analysts '
                    'alike.\n'
                    '\n'
                    'As of now, there ar

[11:writes] Finished step 11 with writes to 3 channels:
- sources_gathered -> ['* BGT 2024-25: Virat Kohli hits 30th Test hundred as India set a target ...\n'
 '  URL: '
 'https://telanganatoday.com/bgt-2024-25-virat-kohli-hits-30th-test-hundred-as-india-set-a-target-of-534-for-australia\n'
 '  Excerpt: Perth: Talismanic batter Virat Kohli smashed his 30th Test '
 "hundred 'and the first of this year' as India set Australia a target of 534 "
 'runs to chase after declaring their second innings at 487/6 in 13...\n'
 '\n'
 '* Virat Kohli Scores 30th Test Century, Surpasses Sir Don ... - Oneindia\n'
 '  URL: '
 'https://www.oneindia.com/sports/cricket/virat-kohli-scores-30th-test-century-surpasses-sir-don-bradman-as-india-set-australia-534-run-targe-3997119.html\n'
 '  Excerpt: India resumed their innings with a 218-run advantage after having '
 'gained control on Day 2 when Australia were dismissed for a meagre 104 in '
 "response to India's 150. Published On November 24, 2024...\n"
 '\n

[12:writes] Finished step 12 with writes to 1 channel:
- running_summary -> ('Virat Kohli has not played any international cricket matches in 2024, as the '
 'year is still ongoing and no official matches have taken place. Therefore, '
 'he has not scored any centuries in 2024 yet. However, according to ESPN '
 "Cricinfo, Virat Kohli's performance in 2024 is expected to be closely "
 'monitored by fans and analysts alike.\n'
 '\n'
 'As of now, there are no match centuries recorded for Virat Kohli in 2024, as '
 'the year has just begun and no official matches have placed. Fans can expect '
 'to see Kohli back on the field soon, looking to add to his impressive '
 'international record.\n'
 '\n'
 "According to recent reports from Cricbuzz, Kohli's team, India, is scheduled "
 'to play a series of matches against Australia in 2024, which could '
 'potentially provide him with opportunities to score centuries and further '
 'boost his international record. Additionally, a report by The In

[13:writes] Finished step 13 with writes to 1 channel:
- search_query -> ("What is Virat Kohli's current international batting record, including his "
 'total number of match centuries and average score, as per ESPN Cricinfo or '
 'other reliable cricket statistics sources?')
[13:checkpoint] State at the end of step 13:
{'research_loop_count': 4,
 'research_topic': 'how many match centuries has virat kohli scored in 2024?',
 'running_summary': 'Virat Kohli has not played any international cricket '
                    'matches in 2024, as the year is still ongoing and no '
                    'official matches have taken place. Therefore, he has not '
                    'scored any centuries in 2024 yet. However, according to '
                    "ESPN Cricinfo, Virat Kohli's performance in 2024 is "
                    'expected to be closely monitored by fans and analysts '
                    'alike.\n'
                    '\n'
                    'As of now, there are no match ce

[14:writes] Finished step 14 with writes to 1 channel:
- running_summary -> ('## Summary\n'
 '\n'
 'Virat Kohli has not played any international cricket matches in 2024, as the '
 'year is still ongoing and no official matches have taken place. Therefore, '
 'he has not scored any centuries in 2024 yet. However, according to ESPN '
 "Cricinfo, Virat Kohli's performance in 2024 is expected to be closely "
 'monitored by fans and analysts alike.\n'
 '\n'
 'As of now, there are no match centuries recorded for Virat Kohli in 2024, as '
 'the year has just begun and no official matches have placed. Fans can expect '
 'to see Kohli back on the field soon, looking to add to his impressive '
 'international record.\n'
 '\n'
 "According to recent reports from Cricbuzz, Kohli's team, India, is scheduled "
 'to play a series of matches against Australia in 2024, which could '
 'potentially provide him with opportunities to score centuries and further '
 'boost his international record. Additional

In [ ]:
# !pip install grandalf
# print("Graph Structure:")
# print(graph.get_graph
# ().draw_ascii())
def deduplicate_and_format_sources(search_response, max_tokens_per_source=1000, include_raw_content=True):
    """Handle malformed API responses gracefully"""
    try:
        if isinstance(search_response, dict):
            sources_list = search_response.get("results", [])
        elif isinstance(search_response, list):
            sources_list = []
            for response in search_response:
                if isinstance(response, dict):
                    sources_list.extend(response.get("results", []))
                else:
                    print(f"Skipping invalid response item: {response}")
        else:
            raise ValueError("Invalid search response format")

        # Filter out non-dictionary items
        sources_list = [s for s in sources_list if isinstance(s, dict)]
        
        # Rest of your existing deduplication logic...
        # [Keep the remainder of your original code here]
        
    except Exception as e:
        print(f"Error processing sources: {e}")
        return "No valid sources found"

def format_sources(results):
    """Handle missing fields gracefully and format multiple sources"""
    sources = []
    for res in results:
        if isinstance(res, dict):
            url = res.get("url", "No URL available")
            title = res.get("title", "Untitled Source")
            content = res.get("content", "")[:200] + "..."  # Show first 200 chars of content
            sources.append(f"* {title}\n  URL: {url}\n  Excerpt: {content}\n")
        else:
            print(f"Skipping invalid result: {res}")
    return "\n".join(sources)


In [ ]:
print("FINAL SUMMARY:\n", summary["running_summary"])
print("\nSOURCES:\n", "\n".join(summary["sources_gathered"]))

FINAL SUMMARY:
 {'final_summary': "Virat Kohli has scored 2 centuries in 2024: one in international Test cricket and one in the Indian Premier League (IPL). According to detailed year-wise breakdowns from reputable cricket statistics sources, Kohli scored a Test century against Australia in Perth and an IPL century (113* vs Rajasthan Royals) in 2024. A search of authoritative sources such as The Cricket Panda and Wikipedia confirms that Kohli's total for 2024 stands at 1 Test century and 1 IPL century. In fact, The Cricket Panda provides a year-by-year table showing 1 century in 2024, and their IPL century list confirms his century in the 2024 season."}

SOURCES:
 https://www.cricbuzz.com/profiles/413/virat-kohli  |  Virat Kohli Player Profile - Cricbuzz
  Excerpt: Detailed stats and records of Virat Kohli including centuries scored in 2024 across formats, highlighting his consistency and match-winning performances.
https://www.espncricinfo.com/player/virat-kohli-253802  |  Virat Kohli

In [16]:
from langgraph.graph import Graph
from IPython.display import display, HTML

class GraphVisualizer:
    def __init__(self, graph):
        self.graph = graph
        self.nodes = list(graph.nodes)
        self.edges = list(graph.edges)
        
    def _generate_html(self):
        html = """
        <style>
            .graph-container { display: flex; flex-direction: column; gap: 20px; }
            .node { padding: 10px; border: 2px solid #4CAF50; border-radius: 5px; margin: 5px; }
            .active { background-color: #4CAF50; color: white; }
            .edge { height: 2px; background: #ccc; margin: 0 10px; position: relative; }
            .edge:after { content: '→'; position: absolute; right: -10px; top: -8px; }
        </style>
        <div class="graph-container" id="graph-viz"></div>
        <script>
            function updateGraph(activeNode) {
                const container = document.getElementById('graph-viz');
                container.innerHTML = '';
                
                // Create nodes
                const nodes = %s;
                nodes.forEach(node => {
                    const div = document.createElement('div');
                    div.className = `node ${activeNode === node ? 'active' : ''}`;
                    div.textContent = node;
                    container.appendChild(div);
                    
                    // Add edge after each node except last
                    if (node !== nodes[nodes.length - 1]) {
                        const edge = document.createElement('div');
                        edge.className = 'edge';
                        container.appendChild(edge);
                    }
                });
            }
        </script>
        """ % str(self.nodes)
        return html
    
    def show(self, active_node=None):
        display(HTML(self._generate_html()))
        if active_node:
            display(HTML(f"<script>updateGraph('{active_node}')</script>"))

# Usage:
visualizer = GraphVisualizer(graph.get_graph())
visualizer.show()  # Shows static structure

# For real-time updates (call this during execution):
# visualizer.show(active_node="web_research") 

In [ ]:
!pip list | grep -E "langchain|tavily|ollama|streamlit|langgraph|langsmith"

# !pip install --upgrade langgraph pyppeteer
# !pip install pyppeteer langgraph[render]
# !pip install "langgraph[render]" 
# For zsh/bash shells:

# !pip uninstall langgraph -y
# !pip install --force-reinstall "langgraph[render]"
# Run in a Python cell:
# import langgraph.render
# print(langgraph.render.__file__)  # Should show path to render module
# !pyppeteer-install  # Install Chromium for Pyppeteer
# Use quotes to handle the bracket in zsh
# !pip install langgraph"render"

# !pip uninstall langgraph -y
# !pip install "langgraph==0.3.33" "pyppeteer==2.0.0" "urllib3<2.0.0"
# !pip install pillow matplotlib
# !pyppeteer-install

# !export TOKENIZERS_PARALLELISM=false
# # Or add this line to your .bashrc/.zshrc for persistence
# !pip install --upgrade langchain-core>=0.1.7 langchain>=0.1.0 

# !pip install --upgrade langchain-ollama
# !pip install --upgrade pip
# !pip install tavily-python


langchain-core       0.3.59
langchain-ollama     0.3.2
langgraph            0.3.33
langgraph-checkpoint 2.0.25
langgraph-prebuilt   0.1.8
langgraph-sdk        0.1.69
langsmith            0.3.42
ollama               0.4.8
tavily-python        0.7.2
